In [1]:
import os
from typing import Tuple
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision.io.image import read_image
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from data.utils import walk_path

In [2]:
from utils.helper_functions import StreetHazardsDataModule
from data.shift_dataset import LabelFilter

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, TQDMProgressBar
from nets.wrapper import Wrapper

In [4]:
# ds_train, ds_val, ds_test = load_shift_segmentation(
#     "./datasets/SHIFT", 512, 6, LabelFilter("4", -1, 0), LabelFilter("4", -1, 0), "ood_pedestrian", 8, .05)
# print(len(ds_train.dataset), len(ds_val.dataset), len(ds_test.dataset))

In [5]:
# ds_train, ds_val, ds_test = load_shift_ood(
#     "./datasets/SHIFT", 512,
#     "./datasets/COCO2014", 256, 4, LabelFilter("4", -1, 0), LabelFilter("4", 10000, 15000), "ood_pedestrian",
#     .7, .9, True, 3, 1, .05)

In [6]:
# ds_train, ds_val, ds_test = load_streethazards_ood(
#     "./datasets/StreetHazards", 512,
#     "./datasets/COCO2014", 256, 4, "normal",
#     .7, .9, True, 3, 1)

In [7]:
dm = StreetHazardsDataModule(
    "./datasets/StreetHazards", 512, 4, "normal", 8)

In [8]:
model = Wrapper("resnet50", 13, .0001)

In [9]:
ckpt = ModelCheckpoint(
        monitor="val_miou",
        mode="max",
        save_top_k=2,
        filename='{epoch}-{step}-{val_miou:.4f}'
    )

In [10]:
tr = Trainer(default_root_dir="./test", accelerator="cuda", callbacks=[ckpt, LearningRateMonitor(logging_interval="epoch"),
        TQDMProgressBar(refresh_rate=5)], max_epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/martin/outlier-detection/.conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [11]:
from torchsummary import summary

In [12]:
summary(model.model, (3, 512, 512), device="cpu")

AttributeError: 'list' object has no attribute 'size'

In [13]:
tr.validate(model=model, datamodule=dm)

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0:   0%|          | 5/1031 [00:09<31:00,  0.55it/s]

/home/martin/outlier-detection/.conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
